In [2]:
import numpy as np
import pandas as pd
from textwrap import dedent

class MatrixOperationsTool:
    def __init__(self):
        self.matrices = {}
        self.operations_history = []
        
    def display_menu(self):
        """Display the main menu"""
        menu = """
        ╔══════════════════════════════════════════╗
        ║           MATRIX OPERATIONS TOOL         ║
        ╠══════════════════════════════════════════╣
        ║ 1. Input New Matrix                      ║
        ║ 2. Display All Matrices                  ║
        ║ 3. Matrix Addition                       ║
        ║ 4. Matrix Subtraction                    ║
        ║ 5. Matrix Multiplication                 ║
        ║ 6. Matrix Transpose                      ║
        ║ 7. Matrix Determinant                    ║
        ║ 8. Matrix Inverse                        ║
        ║ 9. Element-wise Operations               ║
        ║ 10. Operations History                   ║
        ║ 11. Clear All Matrices                   ║
        ║ 0. Exit                                  ║
        ╚══════════════════════════════════════════╝
        """
        print(dedent(menu))
    
    def input_matrix(self):
        """Allow user to input a matrix"""
        try:
            print("\n" + "═" * 50)
            print("INPUT NEW MATRIX")
            print("═" * 50)
            
            name = input("Enter a name for this matrix (e.g., A, B, C): ").strip().upper()
            if not name:
                print("❌ Matrix name cannot be empty!")
                return
            
            if name in self.matrices:
                print(f"❌ Matrix '{name}' already exists!")
                return
            
            rows = int(input("Enter number of rows: "))
            cols = int(input("Enter number of columns: "))
            
            if rows <= 0 or cols <= 0:
                print("❌ Rows and columns must be positive integers!")
                return
            
            print(f"\nEnter matrix elements for {rows}×{cols} matrix:")
            print("Enter each row separated by spaces, rows separated by newlines")
            print("Example for 2×2:")
            print("1 2")
            print("3 4")
            
            matrix = []
            for i in range(rows):
                while True:
                    try:
                        row_input = input(f"Row {i+1}: ").strip()
                        if not row_input:
                            print("❌ Row cannot be empty!")
                            continue
                            
                        row_elements = list(map(float, row_input.split()))
                        if len(row_elements) != cols:
                            print(f"❌ Expected {cols} elements, got {len(row_elements)}!")
                            continue
                            
                        matrix.append(row_elements)
                        break
                    except ValueError:
                        print("❌ Please enter valid numbers!")
            
            np_matrix = np.array(matrix)
            self.matrices[name] = np_matrix
            self.operations_history.append(f"Created matrix '{name}' with shape {np_matrix.shape}")
            print(f"✅ Matrix '{name}' saved successfully!")
            self.display_matrix(np_matrix, name)
            
        except ValueError:
            print("❌ Please enter valid numbers!")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def display_matrix(self, matrix, name="Matrix"):
        """Display a matrix in a formatted way"""
        print(f"\n{name}:")
        
        # Calculate appropriate width for the box
        if len(matrix.shape) == 1 or matrix.shape[0] == 0:
            width = 10
        else:
            # Estimate width based on number of columns
            width = max(8 * matrix.shape[1], 20)
        
        print("┌" + "─" * width + "┐")
        for i, row in enumerate(matrix):
            row_str = "│ "
            for element in row:
                # Format numbers nicely
                if abs(element - int(element)) < 1e-10:  # Essentially an integer
                    row_str += f"{int(element):8.1f} "
                else:
                    row_str += f"{element:8.3f} "
            row_str += "│"
            print(row_str)
        print("└" + "─" * width + "┘")
        print(f"Shape: {matrix.shape}")
    
    def display_all_matrices(self):
        """Display all stored matrices"""
        if not self.matrices:
            print("❌ No matrices stored!")
            return
        
        print("\n" + "═" * 50)
        print("STORED MATRICES")
        print("═" * 50)
        
        for name, matrix in self.matrices.items():
            self.display_matrix(matrix, name)
            print()
    
    def matrix_addition(self):
        """Perform matrix addition"""
        self._binary_operation("ADDITION", lambda A, B: A + B, "A + B")
    
    def matrix_subtraction(self):
        """Perform matrix subtraction"""
        self._binary_operation("SUBTRACTION", lambda A, B: A - B, "A - B")
    
    def matrix_multiplication(self):
        """Perform matrix multiplication"""
        self._binary_operation("MULTIPLICATION", lambda A, B: A @ B, "A × B")
    
    def _binary_operation(self, operation_name, operation_func, operation_symbol):
        """Helper function for binary operations"""
        try:
            if len(self.matrices) < 2:
                print(f"❌ Need at least 2 matrices for {operation_name.lower()}!")
                return
            
            print(f"\n{operation_name}")
            print("Available matrices:")
            for name in self.matrices.keys():
                print(f"  {name} (shape: {self.matrices[name].shape})")
            
            matrix1_name = input("Enter first matrix name: ").strip().upper()
            matrix2_name = input("Enter second matrix name: ").strip().upper()
            
            if matrix1_name not in self.matrices or matrix2_name not in self.matrices:
                print("❌ One or both matrices not found!")
                return
            
            A = self.matrices[matrix1_name]
            B = self.matrices[matrix2_name]
            
            result = operation_func(A, B)
            result_name = f"({matrix1_name} {operation_symbol} {matrix2_name})"
            
            print(f"\n✅ Operation successful!")
            print(f"\nMatrix A:")
            self.display_matrix(A, matrix1_name)
            print(f"\nMatrix B:")
            self.display_matrix(B, matrix2_name)
            print(f"\nResult {operation_symbol}:")
            self.display_matrix(result, result_name)
            
            # Ask if user wants to save result
            save = input("\nSave this result? (y/n): ").strip().lower()
            if save == 'y':
                new_name = input("Enter name for result: ").strip().upper()
                if new_name:
                    self.matrices[new_name] = result
                    print(f"✅ Result saved as '{new_name}'")
            
            self.operations_history.append(f"{operation_name}: {matrix1_name} {operation_symbol} {matrix2_name}")
            
        except ValueError as e:
            print(f"❌ Dimension mismatch! {e}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def matrix_transpose(self):
        """Perform matrix transpose"""
        try:
            if not self.matrices:
                print("❌ No matrices stored!")
                return
            
            print("\nMATRIX TRANSPOSE")
            print("Available matrices:")
            for name in self.matrices.keys():
                print(f"  {name} (shape: {self.matrices[name].shape})")
            
            matrix_name = input("Enter matrix name: ").strip().upper()
            
            if matrix_name not in self.matrices:
                print("❌ Matrix not found!")
                return
            
            matrix = self.matrices[matrix_name]
            transpose = matrix.T
            
            print(f"\n✅ Transpose calculated!")
            print(f"\nOriginal Matrix:")
            self.display_matrix(matrix, matrix_name)
            print(f"\nTranspose:")
            self.display_matrix(transpose, f"{matrix_name}ᵀ")
            
            # Ask if user wants to save result
            save = input("\nSave transpose? (y/n): ").strip().lower()
            if save == 'y':
                new_name = input("Enter name for transpose: ").strip().upper()
                if new_name:
                    self.matrices[new_name] = transpose
                    print(f"✅ Transpose saved as '{new_name}'")
            
            self.operations_history.append(f"TRANSPOSE: {matrix_name} -> {matrix_name}ᵀ")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def matrix_determinant(self):
        """Calculate matrix determinant"""
        try:
            if not self.matrices:
                print("❌ No matrices stored!")
                return
            
            print("\nMATRIX DETERMINANT")
            print("Available matrices:")
            for name, matrix in self.matrices.items():
                if matrix.shape[0] == matrix.shape[1]:
                    print(f"  {name} (shape: {matrix.shape}) - SQUARE")
                else:
                    print(f"  {name} (shape: {matrix.shape}) - NOT SQUARE")
            
            matrix_name = input("Enter matrix name: ").strip().upper()
            
            if matrix_name not in self.matrices:
                print("❌ Matrix not found!")
                return
            
            matrix = self.matrices[matrix_name]
            
            if matrix.shape[0] != matrix.shape[1]:
                print("❌ Determinant can only be calculated for square matrices!")
                return
            
            determinant = np.linalg.det(matrix)
            
            print(f"\n✅ Determinant calculated!")
            print(f"\nMatrix:")
            self.display_matrix(matrix, matrix_name)
            print(f"\nDeterminant: {determinant:.6f}")
            
            # Additional info
            if abs(determinant) < 1e-10:
                print("📝 This matrix is SINGULAR (non-invertible)")
            else:
                print("📝 This matrix is INVERTIBLE")
            
            self.operations_history.append(f"DETERMINANT: det({matrix_name}) = {determinant:.6f}")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def matrix_inverse(self):
        """Calculate matrix inverse"""
        try:
            if not self.matrices:
                print("❌ No matrices stored!")
                return
            
            print("\nMATRIX INVERSE")
            print("Available matrices:")
            for name, matrix in self.matrices.items():
                if matrix.shape[0] == matrix.shape[1]:
                    print(f"  {name} (shape: {matrix.shape}) - SQUARE")
                else:
                    print(f"  {name} (shape: {matrix.shape}) - NOT SQUARE")
            
            matrix_name = input("Enter matrix name: ").strip().upper()
            
            if matrix_name not in self.matrices:
                print("❌ Matrix not found!")
                return
            
            matrix = self.matrices[matrix_name]
            
            if matrix.shape[0] != matrix.shape[1]:
                print("❌ Inverse can only be calculated for square matrices!")
                return
            
            determinant = np.linalg.det(matrix)
            if abs(determinant) < 1e-10:  # Check if matrix is singular
                print("❌ Matrix is singular (determinant ≈ 0), inverse does not exist!")
                return
            
            inverse = np.linalg.inv(matrix)
            
            print(f"\n✅ Inverse calculated!")
            print(f"\nOriginal Matrix:")
            self.display_matrix(matrix, matrix_name)
            print(f"\nInverse Matrix:")
            self.display_matrix(inverse, f"{matrix_name}⁻¹")
            
            # Verify: A × A⁻¹ should be identity
            identity_approx = matrix @ inverse
            print(f"\nVerification (A × A⁻¹ ≈ I):")
            self.display_matrix(identity_approx, "A × A⁻¹")
            
            # Ask if user wants to save result
            save = input("\nSave inverse? (y/n): ").strip().lower()
            if save == 'y':
                new_name = input("Enter name for inverse: ").strip().upper()
                if new_name:
                    self.matrices[new_name] = inverse
                    print(f"✅ Inverse saved as '{new_name}'")
            
            self.operations_history.append(f"INVERSE: {matrix_name} -> {matrix_name}⁻¹")
            
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def elementwise_operations(self):
        """Perform element-wise operations"""
        try:
            print("\nELEMENT-WISE OPERATIONS")
            print("1. Scalar Multiplication")
            print("2. Scalar Addition")
            print("3. Element-wise Multiplication (Hadamard)")
            print("4. Element-wise Division")
            
            choice = input("Choose operation (1-4): ").strip()
            
            if choice == '1':
                self._scalar_operation("MULTIPLICATION", lambda m, s: m * s, "×")
            elif choice == '2':
                self._scalar_operation("ADDITION", lambda m, s: m + s, "+")
            elif choice == '3':
                self._binary_operation("ELEMENT-WISE MULTIPLICATION", lambda A, B: A * B, "⊙")
            elif choice == '4':
                self._binary_operation("ELEMENT-WISE DIVISION", lambda A, B: A / B, "÷")
            else:
                print("❌ Invalid choice!")
                
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def _scalar_operation(self, operation_name, operation_func, operation_symbol):
        """Helper function for scalar operations"""
        try:
            if not self.matrices:
                print("❌ No matrices stored!")
                return
            
            print(f"\nSCALAR {operation_name}")
            print("Available matrices:")
            for name in self.matrices.keys():
                print(f"  {name} (shape: {self.matrices[name].shape})")
            
            matrix_name = input("Enter matrix name: ").strip().upper()
            
            if matrix_name not in self.matrices:
                print("❌ Matrix not found!")
                return
            
            scalar = float(input("Enter scalar value: "))
            matrix = self.matrices[matrix_name]
            
            result = operation_func(matrix, scalar)
            
            print(f"\n✅ Operation successful!")
            print(f"\nOriginal Matrix:")
            self.display_matrix(matrix, matrix_name)
            print(f"\nScalar: {scalar}")
            print(f"\nResult ({operation_symbol} {scalar}):")
            self.display_matrix(result, f"{matrix_name} {operation_symbol} {scalar}")
            
            # Ask if user wants to save result
            save = input("\nSave result? (y/n): ").strip().lower()
            if save == 'y':
                new_name = input("Enter name for result: ").strip().upper()
                if new_name:
                    self.matrices[new_name] = result
                    print(f"✅ Result saved as '{new_name}'")
            
            self.operations_history.append(f"SCALAR {operation_name}: {matrix_name} {operation_symbol} {scalar}")
            
        except ValueError:
            print("❌ Please enter a valid number!")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    def show_operations_history(self):
        """Display operations history"""
        if not self.operations_history:
            print("❌ No operations performed yet!")
            return
        
        print("\n" + "═" * 50)
        print("OPERATIONS HISTORY")
        print("═" * 50)
        
        for i, operation in enumerate(self.operations_history, 1):
            print(f"{i:2d}. {operation}")
    
    def clear_matrices(self):
        """Clear all stored matrices"""
        if not self.matrices:
            print("❌ No matrices to clear!")
            return
        
        confirm = input("Are you sure you want to clear ALL matrices? (y/n): ").strip().lower()
        if confirm == 'y':
            self.matrices.clear()
            self.operations_history.append("CLEARED ALL MATRICES")
            print("✅ All matrices cleared!")
        else:
            print("Operation cancelled.")
    
    def run(self):
        """Main application loop"""
        print("🚀 Welcome to Matrix Operations Tool!")
        print("Perform various matrix operations using NumPy")
        
        while True:
            self.display_menu()
            
            try:
                choice = input("\nEnter your choice (0-11): ").strip()
                
                if choice == '0':
                    print("👋 Thank you for using Matrix Operations Tool!")
                    break
                elif choice == '1':
                    self.input_matrix()
                elif choice == '2':
                    self.display_all_matrices()
                elif choice == '3':
                    self.matrix_addition()
                elif choice == '4':
                    self.matrix_subtraction()
                elif choice == '5':
                    self.matrix_multiplication()
                elif choice == '6':
                    self.matrix_transpose()
                elif choice == '7':
                    self.matrix_determinant()
                elif choice == '8':
                    self.matrix_inverse()
                elif choice == '9':
                    self.elementwise_operations()
                elif choice == '10':
                    self.show_operations_history()
                elif choice == '11':
                    self.clear_matrices()
                else:
                    print("❌ Invalid choice! Please enter 0-11.")
                
                input("\nPress Enter to continue...")
                
            except KeyboardInterrupt:
                print("\n\n👋 Program interrupted. Goodbye!")
                break
            except Exception as e:
                print(f"❌ Unexpected error: {e}")
                input("Press Enter to continue...")

# Create a simple test function to demonstrate
def test_matrix_tool():
    """Test the matrix operations with sample data"""
    tool = MatrixOperationsTool()
    
    # Create some sample matrices
    A = np.array([[1, 2], [3, 4]])
    B = np.array([[5, 6], [7, 8]])
    
    tool.matrices['A'] = A
    tool.matrices['B'] = B
    
    print("Sample matrices created:")
    tool.display_all_matrices()
    
    print("\nTesting addition:")
    result = A + B
    print("A + B =")
    tool.display_matrix(result)
    
    print("\nTesting multiplication:")
    result = A @ B
    print("A × B =")
    tool.display_matrix(result)
    
    print("\nTesting determinant:")
    det = np.linalg.det(A)
    print(f"det(A) = {det}")

def main():
    """Main function to run the application"""
    try:
        tool = MatrixOperationsTool()
        tool.run()
    except Exception as e:
        print(f"❌ Application error: {e}")

if __name__ == "__main__":
    main()

🚀 Welcome to Matrix Operations Tool!
Perform various matrix operations using NumPy

╔══════════════════════════════════════════╗
║           MATRIX OPERATIONS TOOL         ║
╠══════════════════════════════════════════╣
║ 1. Input New Matrix                      ║
║ 2. Display All Matrices                  ║
║ 3. Matrix Addition                       ║
║ 4. Matrix Subtraction                   ║
║ 5. Matrix Multiplication                 ║
║ 6. Matrix Transpose                     ║
║ 7. Matrix Determinant                   ║
║ 8. Matrix Inverse                       ║
║ 9. Element-wise Operations              ║
║ 10. Operations History                  ║
║ 11. Clear All Matrices                  ║
║ 0. Exit                                ║
╚══════════════════════════════════════════╝




Enter your choice (0-11):  1



══════════════════════════════════════════════════
INPUT NEW MATRIX
══════════════════════════════════════════════════


Enter a name for this matrix (e.g., A, B, C):  A
Enter number of rows:  2
Enter number of columns:  2



Enter matrix elements for 2×2 matrix:
Enter each row separated by spaces, rows separated by newlines
Example for 2×2:
1 2
3 4


...


✅ Operation successful!

Matrix A:

A:
┌────────────────────┐
│      1.0      2.0 │
│      3.0      4.0 │
└────────────────────┘
Shape: (2, 2)

Matrix B:

B:
┌────────────────────┐
│      4.0      5.0 │
│      6.0      7.0 │
└────────────────────┘
Shape: (2, 2)

Result A + B:

(A A + B B):
┌────────────────────┐
│      5.0      7.0 │
│      9.0     11.0 │
└────────────────────┘
Shape: (2, 2)



Save this result? (y/n):  y
Enter name for result:  

Press Enter to continue... 



╔══════════════════════════════════════════╗
║           MATRIX OPERATIONS TOOL         ║
╠══════════════════════════════════════════╣
║ 1. Input New Matrix                      ║
║ 2. Display All Matrices                  ║
║ 3. Matrix Addition                       ║
║ 4. Matrix Subtraction                   ║
║ 5. Matrix Multiplication                 ║
║ 6. Matrix Transpose                     ║
║ 7. Matrix Determinant                   ║
║ 8. Matrix Inverse                       ║
║ 9. Element-wise Operations              ║
║ 10. Operations History                  ║
║ 11. Clear All Matrices                  ║
║ 0. Exit                                ║
╚══════════════════════════════════════════╝




Enter your choice (0-11):  0


👋 Thank you for using Matrix Operations Tool!
